In [ ]:
# Importing Libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy as sp
import numpy as np
import pandas as pd
import time
import datetime
import os
import sys
import pyvisa
import threading
import ipywidgets
import nidaqmx
from scipy import signal
from collections import deque
from IPython.display import clear_output, display

print(f"All Libraries imported successfully at {datetime.datetime.now()}")

In [ ]:
# Connecting to Instruments
rm = pyvisa.ResourceManager()
sr830_1 = rm.open_resource('GPIB1::6::INSTR')
sr830_2 = rm.open_resource('GPIB1::7::INSTR')
sr830_3 = rm.open_resource('GPIB1::8::INSTR')
sr860_1 = rm.open_resource('GPIB1::10::INSTR')
sr860_2 = rm.open_resource('GPIB1::11::INSTR')
sr860_3 = rm.open_resource('GPIB1::12::INSTR')
lakeshore = rm.open_resource("GPIB1::5::INSTR")
ami430 = rm.open_resource('TCPIP::192.168.1.30::7180::SOCKET', timeout=5000, write_termination='\n', read_termination='\n')

print(f"Instruments connected successfully at {datetime.datetime.now()}")

In [ ]:
"""
General functions (Not related to any specific instrument)
"""

# Function for defining the stop button click event handler
def on_stop_clicked(b):
    stop_event.set()
    status_btn.description = "Stopped"
    status_btn.button_style = "danger"
    stop_btn.disabled = True
    stop_btn.close()

# Function for creating path for saving file in a specific folder
def save_file(folder,filename): # It will add date prefix to filename
    """
    It will create the folder if it does not exist 
    and return the full path for saving file in that folder.
    """
    current_path = os.getcwd()
    folder_path = os.path.join(current_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    filename = f"{datetime.datetime.now().strftime('%y%m%d')}_{filename}"
    file_path = os.path.join(folder_path, filename)
    return file_path

# Function for creating path for loading file from a specific folder
def load_file(folder,filename): # It will not add date prefix to filename
    """
    Please provide the proper filename with extension.
    It will return the full path for loading file from that folder.
    """
    current_path = os.getcwd()
    file_path = os.path.join(current_path, folder, filename)
    return file_path

# Function for creating array for sweeping
def sweep(vmin, vmax, vstep, vmode): # Modes: up, down, updown, downup, round-up, round-down
    if vmode =='up': # lower to higher value
        sweep = np.arange(vmin,vmax+vstep,vstep)
    elif vmode =='down': # higher to lower value
        sweep = np.arange(vmax,vmin-vstep,-vstep)
    elif vmode =='updown': # lower to higher to lower value
        sweep = np.arange(vmin,vmax+vstep,vstep)
        sweep = np.append(sweep,np.arange(vmax-vstep,vmin-vstep,-vstep))
    elif vmode =='downup': # higher to lower to higher value
        sweep = np.arange(vmax,vmin-vstep,-vstep)
        sweep = np.append(sweep,np.arange(vmin+vstep,vmax+vstep,vstep))
    elif vmode =='round-up': # zero to higher to lower to zero value
        sweep = np.arange(0,vmax+vstep,vstep)
        sweep = np.append(sweep,np.arange(vmax-vstep,vmin-vstep,-vstep))
        sweep = np.append(sweep,np.arange(vmin+vstep,vstep,vstep))
    elif vmode =='round-down': # zero to lower to higher to zero value
        sweep = np.arange(0,vmin-vstep,-vstep)
        sweep = np.append(sweep,np.arange(vmin+vstep,vmax+vstep,vstep))
        sweep = np.append(sweep,np.arange(vmax-vstep,-vstep,-vstep))
    else:
        print('Error: Invalid Sweep Mode')
        sys.exit()
    return sweep

# Function for converting current values to voltage values
def ItoV(i_min,i_max,i_step,R_series):
    '''returns v_min,v_max,v_step'''
    if R_series == 0:
        R_series = 1
    v_min=i_min*R_series
    v_max=i_max*R_series
    v_step=i_step*R_series
    return v_min,v_max,v_step

# Function for converting voltage values to current values
def VtoI(v_min,v_max,v_step,R_series):
    '''returns i_min,i_max,i_step'''
    if R_series == 0:
        R_series = 1
    i_min=v_min/R_series
    i_max=v_max/R_series
    i_step=v_step/R_series
    return i_min,i_max,i_step

# Function for converting XYRT to dVdI
def calculate_dVdI(X,Y,R,I,Gain):
    dVdI_x=X/(I*Gain)
    dVdI_y=Y/(I*Gain)
    dVdI_r=R/(I*Gain)
    return dVdI_x,dVdI_y,dVdI_r

print(f"All Helper functions are defined successfully at {datetime.datetime.now()}")

In [ ]:
"""
AMI430 Related Functions
"""
# Function to get AMI430 status
def get_ami_status(ami430): # Returns AMI430 state as string
    dummy = ami430.query_ascii_values('STATE?')
    dummy = ami430.query_ascii_values('STATE?')
    ami_state_number = ami430.query_ascii_values('STATE?')
    ami_state_list = ["RAMPING to target", 
                      "HOLDING at target", 
                      "PAUSED", 
                      "Ramping in MANUAL UP mode", 
                      "Ramping in MANUAL DOWN mode", 
                      "ZEROING CURRENT", 
                      "QUENCH DETECTED", 
                      "AT ZERO current", 
                      "Heating persistent switch", 
                      "Cooling persistent switch"]
    ami_state = ami_state_list[int(ami_state_number[0])-1]
    return ami_state

# Function to set AMI430 field and ramp rate units
def set_ami_unit(field_unit, rate_unit, ami430): # Tesla/kiloGauss and per_minute/per_second
    '''
    Tesla or kiloGauss for field_unit. per_minute or per_second for rate_unit.
    1 Tesla = 10 kiloGauss
    '''
    try:
        if field_unit == 'Tesla':
            field_unit_code = 1
        elif field_unit == 'kiloGauss':
            field_unit_code = 0
        if rate_unit == 'per_minute':
            rate_unit_code = 1
        elif rate_unit == 'per_second':
            rate_unit_code = 0
        ami430.write(f"CONFigure:FIELD:UNITS {field_unit_code}")
        ami430.write(f"CONFigure:RAMP:RATE:UNITS {rate_unit_code}")
        print(f'Units set to {field_unit} {rate_unit}')
    except:
        print('Error: Unable to set units. Check AMI430 and retry.')
        sys.exit()

# Function to set AMI430 ramp rate
def set_ami_ramp_rate(rate, ami430): # Set ramp rate in units defined earlier
    """
    Maximum ramp rate for 0-5 Tesla is 1mT/s (60 mT/min)
    Maximum ramp rate for 5-7 Tesla is 0.5mT/s (30 mT/min)
    Maximum ramp rate for 7-9 Tesla is 0.25mT/s (15 mT/min)
    """
    try:
        ami430.write('CONFigure:RAMP:RATE:SEGments 1')
        ami430.write(f'CONFigure:RAMP:RATE:FIELD 1,{rate},9')
    except:
        pass

# Function to set AMI430 magnetic field
def set_ami_field(B_set, ami430): # Set target magnetic field and start ramping
    '''arguments: B_set - target magnetic field, ami430 - AMI430 instrument object'''
    try:
        dummy = ami430.query_ascii_values('STATE?')[0]
        dummy = ami430.query_ascii_values('STATE?')[0]
        ami_state_number = ami430.query_ascii_values('STATE?')[0]
        if ami_state_number in [1, 2, 3]:
            ami430.write(f'CONFigure:FIELD:TARget {B_set}')
            ami430.write(f'RAMP')
    except:
        pass

# Function for logging the magnetic field from AMI430
def log_ami430(ami430): # Temeperature is dummy value here atleast for now
    try:
        """
        Step 1: Create a log file if does not exist
        """
        logfile = os.path.join(os.getcwd(),'Magnet Log',f'{datetime.datetime.now().strftime("%y%m%d")}_AMI430_MagnetLog.log')
        if not os.path.exists(logfile):
            columns = ['Datetime','State','Temperature_K','Field_T','Vmagnet_V','Vsupply_V']
            pd.DataFrame(columns=columns).to_csv(logfile, index=False)
        """
        Step 2: Read parameters from AMI430
        """
        t = datetime.datetime.now().strftime('%y-%m-%d %H:%M:%S')
        dummy = ami430.query_ascii_values('STATE?')[0]
        dummy = ami430.query_ascii_values('STATE?')[0]
        ami_state_number = ami430.query_ascii_values('STATE?')[0]
        Temp = np.random.randint(0, 100)
        dummy = ami430.query_ascii_values('FIELD:MAGnet?')[0]
        dummy = ami430.query_ascii_values('FIELD:MAGnet?')[0]
        Field = ami430.query_ascii_values('FIELD:MAGnet?')[0]
        dummy = ami430.query_ascii_values('VOLTage:MAGnet?')[0]
        dummy = ami430.query_ascii_values('VOLTage:MAGnet?')[0]
        V_magnet = ami430.query_ascii_values('VOLTage:MAGnet?')[0]
        dummy = ami430.query_ascii_values('VOLTage:SUPPly?')[0]
        dummy = ami430.query_ascii_values('VOLTage:SUPPly?')[0]
        V_supply = ami430.query_ascii_values('VOLTage:SUPPly?')[0]
        row = {"Datetime": t, "State": ami_state_number, "Temperature_K": Temp, "Field_T": Field, "Vmagnet_V": V_magnet, "Vsupply_V": V_supply}
        pd.DataFrame([row]).to_csv(logfile, mode='a', header=False, index=False)
        time.sleep(1)
    except:
        pass

print(f"AMI430 Helper functions are defined successfully at {datetime.datetime.now()}")

In [ ]:
"""
Lakeshore Related Functions
"""
# Function to read temperature from lakeshore via gpib
def readtemp_lakeshore(connection,address, channel): # Read the temperature from lakeshore channel via GPIB, Serial or from log file
    """
    For GPIB connection, adrress argument can be any sting.
    For Serial connection, address agrument is COM port number (integer).
    For LOG connection, address argument is cryostat name (lowercase string).
    """
    if connection == 'GPIB':
        try:
            temp = address.query_ascii_values(f"RDGK? {channel}")[0]
        except Exception:
            temp = np.nan
    if connection == 'SERIAL':
        try:
            import serial
            COM = f'COM{COM}'
            device_serial=serial.Serial(COM,
                                        baudrate=9600,
                                        bytesize=serial.SEVENBITS,
                                        stopbits=serial.STOPBITS_ONE,
                                        parity=serial.PARITY_ODD,
                                        timeout=5)
            device_serial.write(f'RDGK? {channel}\n'.encode('utf-8'))
            temp=float(device_serial.readline(100).decode('utf-8').strip())
            device_serial.close()
        except Exception:
            temp = np.nan
    if connection == 'LOG':
        try:
            if address == 'fincryo': # For Fincryo
                folder="\\\DESKTOP-DQE01NH\Logfiles\\"+str(datetime.now().strftime("%y-%m-%d"))
                file=f"CH{int(channel)} T "+str(datetime.now().strftime("%y-%m-%d"))+".log"
            with open(os.path.join(folder, file), "rb") as f:
                f.seek(-2, 2)  # go to the end of file
                while f.read(1) != b"\n":
                    f.seek(-2, 1)
                last_line = f.readline().decode().strip()
            temp = float(last_line.split(',')[2])
        except Exception:
            temp = np.nan
    return temp

# Functions to set heater parameters
def set_heater(connection,address,mode, range, P, I, D): # Set heater parameters
    '''
    Set heater parameters:
    mode: Heater mode - "Closed Loop PID","Zone Tuning","Open Loop","Off"
    range: Heater range - "off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"
    P, I, D: PID parameters (only for Closed Loop PID mode)
    '''
    if connection == 'GPIB':
        heater_range_list = ["off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"]
        heater_mode_list = ["Closed Loop PID","Zone Tuning","Open Loop","Off"]
        if mode in heater_mode_list:
            mode_index = heater_mode_list.index(mode) + 1
            address.write(f'CMODE {mode_index}')
        else:
            print('Error: Invalid Heater Mode')
            sys.exit()
        if mode == "Closed Loop PID":
            if range in heater_range_list:
                range_index = heater_range_list.index(range)
                address.write(f'HTRRNG {range_index}')
            else:
                print('Error: Invalid Heater Range')
                sys.exit()
            address.write(f'PID {P},{I},{D}')
        print('Heater settings updated successfully')

# Function to get heater status        
def get_heater_status(connection,address): # Returns heater status as dictionary
    if connection == 'GPIB':
        # Function to read heater mode
        def read_heater_mode(address):
            n = address.query_ascii_values('CMODE?')[0]
            heater_mode_list = ["Closed Loop PID","Zone Tuning","Open Loop","Off"]
            val = heater_mode_list[int(n)-1]
            return val
        # Function to read heater range
        def read_heater_range(address):
            n = address.query_ascii_values('HTRRNG?')[0]
            heater_range_list = ["off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"]
            val = heater_range_list[int(n)]
            return val
        # Function to read PID parameters
        def read_pid_params(address):
            params = address.query_ascii_values('PID?')
            return params
        # Reading all heater status
        mode = read_heater_mode(address)
        range = read_heater_range(address)
        P, I, D = read_pid_params(address)
        status = {
            "Mode": mode,
            "Range": range,
            "P": P,
            "I": I,
            "D": D
        }
    return status

print(f"Lakeshore Helper functions are defined successfully at {datetime.datetime.now()}")

In [ ]:
"""
SR830/SR860 Related Functions
"""
# Function to measure X, Y, R, T from SR830
def measure_SR830(address):
    X_comp=address.query_ascii_values('OUTP? 1')[0]
    Y_comp=address.query_ascii_values('OUTP? 2')[0]
    R_comp=address.query_ascii_values('OUTP? 3')[0]
    T_comp=address.query_ascii_values('OUTP? 4')[0]
    return X_comp,Y_comp,R_comp,T_comp

# Function to measure X, Y, R, T from SR860
def measure_SR860(address):
    X_comp=address.query_ascii_values('OUTP? 0')[0]
    Y_comp=address.query_ascii_values('OUTP? 1')[0]
    R_comp=address.query_ascii_values('OUTP? 2')[0]
    T_comp=address.query_ascii_values('OUTP? 3')[0]
    return X_comp,Y_comp,R_comp,T_comp

# Function for sweeping frequency in Lock-in Amplifier SR830 and SR860
def frequency_ramp(fset,fsubstep,instrument,address):
    if instrument == 'SR860':
        if fset < 1e-3:
            fset = 1e-3
        elif fset > 500e3:
            print('Frequency setpoint should be between 1 mHz to 500 kHz')
            sys.exit()
        fnow=address.query_ascii_values('FREQ?')[0] # reading current value
        N=abs(fset-fnow)/fsubstep # dividing the total voltage range into small steps
        for i in np.linspace(fnow,fset,int(N)):
            address.write(f'FREQ {i}')
    elif instrument == 'SR830':
        if fset < 1e-3:
            fset = 1e-3
        elif fset > 102e3:
            print('Frequency setpoint should be between 1 mHz to 102 kHz')
            sys.exit()
        fnow=address.query_ascii_values('FREQ?')[0] # reading current value
        N=abs(fset-fnow)/fsubstep # dividing the total voltage range into small steps
        for i in np.linspace(fnow,fset,int(N)):
            address.write(f'FREQ {i}')
    else:
        print('Error: Invalid Instrument')
        sys.exit()

# Function for ramping vsine in small steps for SR830 and SR860
def vsine_ramp(vset,vsubstep,step_time,instrument,address):
    if instrument == 'SR830':
        if vset < 0.004:
            vset = 0.004
        elif vset > 5:
            print('Voltage setpoint should be between 4mV to 5 V')
            sys.exit()
        vnow=address.query_ascii_values('SLVL?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f'SLVL {i}')
            time.sleep(step_time)
    elif instrument == 'SR860':
        if vset < 1e-9:
            vset = 1e-9
        elif vset > 2:
            print('Voltage setpoint should be between 1 nV to 2 V')
            sys.exit()
        vnow=address.query_ascii_values('SLVL?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f'SLVL {i}')
            time.sleep(step_time)
    else:
        print('Error: Invalid Instrument')
        sys.exit()

# Function for ramping vsine to zero for SR830 and SR860
def vsine_ramp_zero(vsubstep,step_time,instrument,address):
    vsine_ramp(0,vsubstep,step_time,instrument,address)

In [ ]:
# Program for logging AMI magnet parameters
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn, stop_btn)
stop_btn.on_click(on_stop_clicked)
# Measurement Loop in a separate thread
def ami_logging_loop():
    while not stop_event.is_set():
        log_ami430(ami430)

threading.Thread(target=ami_logging_loop, daemon=True).start()

In [ ]:
# Program for live plotting AMI magnet log
def live_plot_loop(folder, filename):
    time.sleep(1)
    logfile = os.path.join(os.getcwd(),folder,f'{datetime.datetime.now().strftime("%y%m%d")}_AMI430_MagnetLog.log')
    while not stop_event.is_set():
        df = pd.read_csv(logfile)
        df['Datetime'] = pd.to_datetime(df['Datetime'], format='%y-%m-%d %H:%M:%S', errors='coerce')
        df = df.dropna(subset=['Datetime'])
        clear_output(wait=True)
        fig, axes = plt.subplots(4, 1, figsize=(10, 10), sharex=True)
        axes[0].plot(df['Datetime'], df['Temperature_K'], 'o', markersize=3)
        axes[0].set_ylabel('T (K)')
        axes[1].plot(df['Datetime'], df['Field_T'], 'o', markersize=3)
        axes[1].set_ylabel('B (T)')
        axes[2].plot(df['Datetime'], df['Vmagnet_V'], 'o', markersize=3)
        axes[2].set_ylabel('Vmagnet (V)')
        axes[3].plot(df['Datetime'], df['Vsupply_V'], 'o', markersize=3)
        axes[3].set_ylabel('Vsupply (V)')
        axes[3].set_xlabel('Datetime')
        axes[3].xaxis.set_major_locator(mdates.AutoDateLocator())
        axes[3].xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d\n%H:%M:%S'))
        State = df['State'].iloc[-1]
        fig.suptitle(f'State: {State}', y=0.98)
        for ax in axes:
            ax.grid(True)
        fig.autofmt_xdate()
        display(fig)
        plt.close(fig)
        time.sleep(1)

threading.Thread(target=live_plot_loop, args=('Magnet Log',f'{datetime.datetime.now().strftime("%y%m%d")}_AMI430_MagnetLog.log'), daemon=True).start()

In [ ]:
# Program for R vs B measurement using 1 SR830 and AMI430 
# It will also log AMI430 parameters in background
# Make sure that the correct SR830 is addressed in SR830_1 variable

B_min = 0  # Minimum magnetic field in Tesla
B_max = 4  # Maximum magnetic field in Tesla
B_step = 0.01  # Magnetic field step in Tesla
B_ramp_rate = 0.015  # Ramp rate in Tesla per minute
Direction = 'down'  # Sweep direction: 'up' or 'down'

# Open log file and write header
filename = save_file("251213", "RvsB_4-0T_11mK_100mV_10MOhm_25-31_2probe.dat") # date prefixed filename automatically
if not os.path.exists(filename):
    cols = ["B", "Vin", "Vx1", "Vy1", "Iin", "Rx1", "Ry1"]
    pd.DataFrame(columns=cols).to_csv(filename, index=False)

# Configuring AMI430
set_ami_unit('Tesla', 'per_minute', ami430)
set_ami_ramp_rate(B_ramp_rate, ami430)

# Measurement loop
for B in sweep(B_min, B_max, B_step, Direction):
    try:
        print(f'Setting target field to {B} T.',end=' ')
        set_ami_field(B, ami430)
        set_ami_field(B, ami430)
        set_ami_field(B, ami430)
        dummy = ami430.query_ascii_values('STATE?')[0]
        dummy = ami430.query_ascii_values('STATE?')[0]
        ami_state_number = ami430.query_ascii_values('STATE?')[0]
        while ami_state_number != 2:
            log_ami430(ami430)
            dummy = ami430.query_ascii_values('STATE?')[0]
            dummy = ami430.query_ascii_values('STATE?')[0]
            ami_state_number = ami430.query_ascii_values('STATE?')[0]
        if ami_state_number == 7:
            print('Quench detected! Stopping measurement.')
            sys.exit()
        while ami_state_number == 2:
            print(f'Target field {B} T reached and holding.')
            dummy = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            dummy = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            Field = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            R_series = 10e6 # Series resistance in Ohms
            Gain = 1    # Gain of the pre-amplifier
            Vin = SR830_1.query_ascii_values('SLVL?')[0]
            Iin = Vin / R_series
            Vx1, Vy1, Vr1, Phase1 = measure_SR830(SR830_1)
            Rx1 = Vx1 / (Iin * Gain)
            Ry1 = Vy1 / (Iin * Gain)
            # Saving data to file
            row = {"B": Field, "Vin": Vin, "Vx1": Vx1, "Vy1": Vy1, "Iin": Iin, "Rx1": Rx1, "Ry1": Ry1}
            pd.DataFrame([row]).to_csv(filename, mode='a', header=False, index=False)
            ami_state_number = 1
    except Exception as e:
        pass


In [ ]:
# Program for R vs B measurement using 3 SR830 and AMI430 
# It will also log AMI430 parameters in background
# Make sure that the correct SR830 are addressed in SR830_1, SR830_2, SR830_3 variables

B_min = 0  # Minimum magnetic field in Tesla
B_max = 4  # Maximum magnetic field in Tesla
B_step = 0.01  # Magnetic field step in Tesla
B_ramp_rate = 0.015  # Ramp rate in Tesla per minute
Direction = 'down'  # Sweep direction: 'up' or 'down'

# Open log file and write header
filename = save_file("251213", "RvsB_0-1T_11mK_100mV_10MOhm_at22_V1_22-23_V2_23-24_V3_24-25.dat")
if not os.path.exists(filename):
    cols = ["B", "Vin", "Vx1", "Vy1", "Vx2", "Vy2", "Vx3", "Vy3", "Iin", "Rx1", "Ry1", "Rx2", "Ry2", "Rx3", "Ry3"]
    pd.DataFrame(columns=cols).to_csv(filename, index=False)
    
# Configuring AMI430
set_ami_unit('Tesla', 'per_minute', ami430)
set_ami_ramp_rate(B_ramp_rate, ami430)
# Measurement loop
for B in sweep(B_min, B_max, B_step, Direction):
    try:
        print(f'Setting target field to {B} T.',end=' ')
        set_ami_field(B, ami430)
        set_ami_field(B, ami430)
        set_ami_field(B, ami430)
        dummy = ami430.query_ascii_values('STATE?')[0]
        dummy = ami430.query_ascii_values('STATE?')[0]
        ami_state_number = ami430.query_ascii_values('STATE?')[0]
        while ami_state_number != 2:
            log_ami430(ami430)
            dummy = ami430.query_ascii_values('STATE?')[0]
            dummy = ami430.query_ascii_values('STATE?')[0]
            ami_state_number = ami430.query_ascii_values('STATE?')[0]
        if ami_state_number == 7:
            print('Quench detected! Stopping measurement.')
            sys.exit()
        while ami_state_number == 2:
            print(f'Target field {B} T reached and holding.')
            dummy = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            dummy = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            Field = ami430.query_ascii_values('FIELD:MAGnet?')[0]
            R_series = 10e6 # Series resistance in Ohms
            Gain = 1    # Gain of the pre-amplifier
            Vin = SR830_1.query_ascii_values('SLVL?')[0]
            Iin = Vin / R_series
            Vx1, Vy1, Vr1, Phase1 = measure_SR830(SR830_1)
            Vx2, Vy2, Vr2, Phase2 = measure_SR830(SR830_2)
            Vx3, Vy3, Vr3, Phase3 = measure_SR830(SR830_3)
            Rx1 = Vx1 / (Iin * Gain)
            Ry1 = Vy1 / (Iin * Gain)
            Rx2 = Vx2 / (Iin * Gain)
            Ry2 = Vy2 / (Iin * Gain)
            Rx3 = Vx3 / (Iin * Gain)
            Ry3 = Vy3 / (Iin * Gain)

            row = {"B": Field, "Vin": Vin, "Vx1": Vx1, "Vy1": Vy1, "Vx2": Vx2, "Vy2": Vy2, "Vx3": Vx3, "Vy3": Vy3, 
                "Iin": Iin, "Rx1": Rx1, "Ry1": Ry1, "Rx2": Rx2, "Ry2": Ry2, "Rx3": Rx3, "Ry3": Ry3}
            pd.DataFrame([row]).to_csv(filename, mode='a', header=False, index=False)
            ami_state_number = 1

    except Exception as e:
        pass